# importing libraries

In [ ]:
from osgeo import gdal
import os

# processing black marble data to get nighttime light tiff files

In [ ]:
## List input raster files
product = "VNP46A2_20210831"
os.chdir(f'../data/{product}/')
rasterFiles = os.listdir(os.getcwd())
rasterFiles = [f for f in rasterFiles if f.endswith('.h5')]

In [ ]:
# Loop through each raster file
for i in range(len(rasterFiles)):

    #Get File Name Prefix
    rasterFilePre = rasterFiles[i][:-3]
    print(rasterFilePre)

    # Define output file extension
    fileExtension = "_BBOX.tif"

    ## Open HDF file
    hdflayer = gdal.Open(rasterFiles[i], gdal.GA_ReadOnly)

    # Get the first subdataset
    subhdflayer = hdflayer.GetSubDatasets()[0][0]
    rlayer = gdal.Open(subhdflayer, gdal.GA_ReadOnly)

    #Subset the Long Name
    outputName = subhdflayer[92:]

    # Clean up output name
    outputNameNoSpace = outputName.strip().replace(" ","_").replace("/","_")
    outputNameFinal = outputNameNoSpace + rasterFilePre + fileExtension
    print(outputNameFinal)

    # Define output folder
    outputFolder = f"../data/{product}/tiff/"

    # Define output raster path
    outputRaster = outputFolder + outputNameFinal

    # Get tile numbers from metadata
    HorizontalTileNumber = int(rlayer.GetMetadata_Dict()["HorizontalTileNumber"])
    VerticalTileNumber = int(rlayer.GetMetadata_Dict()["VerticalTileNumber"])
    
    # Calculate bounding box coordinates
    WestBoundCoord = (10*HorizontalTileNumber) - 180
    NorthBoundCoord = 90-(10*VerticalTileNumber)
    EastBoundCoord = WestBoundCoord + 10
    SouthBoundCoord = NorthBoundCoord - 10

    # Define translation options
    EPSG = "-a_srs EPSG:4326" #WGS84

    # Define the georeferencing option text
    translateOptionText = EPSG+" -a_ullr " + str(WestBoundCoord) + " " + str(NorthBoundCoord) + " " + str(EastBoundCoord) + " " + str(SouthBoundCoord)

    # Perform the translation
    translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
    gdal.Translate(outputRaster,rlayer, options=translateoptions)
